In [1]:
import pandas as pd
import numpy as np
from geopy.distance import distance
import pydeck as pdk
import pickle
import requests
import json

In [2]:
model = pickle.load(open('random_forest.pickle', 'rb'))

In [3]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [4]:
new = data.groupby('dock_id').apply(pd.DataFrame.sample, random_state = 5, n=1).reset_index(drop=True)

In [5]:
new1 = new[['dock_id', 'latitude', 'longitude', 'tot_docks']]

In [6]:
new2 = new[['weekday_cluster', 'weekend_cluster']]

In [7]:
month = 12
day = 20
hour = 12

In [8]:
date = pd.to_datetime({'year': [2021], 'month': [month], 'day': [day], 'hour': [hour]})[0]

In [9]:
day_of_week_conversion = {0:2, 1:3, 2:4, 3:5, 4:6, 5:7, 6:1}

In [10]:
sql_day_of_week = day_of_week_conversion[date.dayofweek]

In [11]:
sql_day_of_week

2

In [12]:
month_column = np.ones(len(new), dtype=int) * month
num_day_column = np.ones(len(new), dtype=int) * sql_day_of_week
hour_column = np.ones(len(new), dtype=int) * hour
date_df = pd.DataFrame({'month': month_column, 'num_day': num_day_column, 'hour': hour_column})

In [13]:
date_df

,month,num_day,hour
0,12,2,12
1,12,2,12
2,12,2,12
3,12,2,12
4,12,2,12
...,...,...,...
324,12,2,12
325,12,2,12
326,12,2,12
327,12,2,12


In [14]:
request = requests.get('https://api.openweathermap.org/data/2.5/onecall?lat=40.7812&lon=-73.9665&exclude=minutely,current,daily,alerts&units=metric&appid=404310456b8e1c31228341dd6c95dd04')
response = request.json()
response

{'lat': 40.7812,
 'lon': -73.9665,
 'timezone': 'America/New_York',
 'timezone_offset': -18000,
 'hourly': [{'dt': 1639857600,
   'temp': 5.54,
   'feels_like': 2.53,
   'pressure': 1015,
   'humidity': 87,
   'dew_point': 3.55,
   'uvi': 0.05,
   'clouds': 90,
   'visibility': 10000,
   'wind_speed': 3.99,
   'wind_deg': 129,
   'wind_gust': 7.85,
   'weather': [{'id': 500,
     'main': 'Rain',
     'description': 'light rain',
     'icon': '10d'}],
   'pop': 0.3,
   'rain': {'1h': 0.16}},
  {'dt': 1639861200,
   'temp': 6,
   'feels_like': 3.23,
   'pressure': 1015,
   'humidity': 86,
   'dew_point': 3.84,
   'uvi': 0,
   'clouds': 92,
   'visibility': 10000,
   'wind_speed': 3.74,
   'wind_deg': 125,
   'wind_gust': 7.84,
   'weather': [{'id': 804,
     'main': 'Clouds',
     'description': 'overcast clouds',
     'icon': '04d'}],
   'pop': 0.22},
  {'dt': 1639864800,
   'temp': 6.64,
   'feels_like': 4.25,
   'pressure': 1015,
   'humidity': 85,
   'dew_point': 4.3,
   'uvi': 0,
  

In [15]:
temp = response['hourly'][22]['temp']
rain = round(response['hourly'][22]['pop'])

In [16]:
weather_df = pd.DataFrame.from_dict(response['hourly'])[['dt', 'temp','pop']]
weather_df['dt'] = pd.to_datetime(weather_df['dt'], unit = 's')

In [17]:
weather_df = weather_df[weather_df['dt'] == date]

In [18]:
temp = weather_df['temp'].iloc[0]
rain = round(weather_df['pop'].iloc[0])

In [19]:
temp_column = np.ones(len(new), dtype=int) * temp
rain_column = np.ones(len(new), dtype=int) * rain

In [20]:
new1['tmp'] = temp_column
new1['rain'] = rain_column

<ipython-input-20-6b2e212f92bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['tmp'] = temp_column
<ipython-input-20-6b2e212f92bf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new1['rain'] = rain_column


In [21]:
day_data = pd.concat([new1, new2], axis = 1)

In [22]:
day_data

,dock_id,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,83,40.683826,-73.976323,62,-1.01,0,2,3
1,119,40.696089,-73.978034,19,-1.01,0,2,0
2,120,40.686768,-73.959282,19,-1.01,0,1,0
3,143,40.692395,-73.993379,24,-1.01,0,2,3
4,144,40.698399,-73.980689,19,-1.01,0,2,0
...,...,...,...,...,...,...,...,...
324,3505,40.805726,-73.936322,33,-1.01,0,2,2
325,3506,40.801307,-73.939817,29,-1.01,0,0,2
326,3507,40.804555,-73.939686,36,-1.01,0,0,2
327,3509,40.801194,-73.950074,31,-1.01,0,2,0


In [23]:
final_df = pd.concat([date_df, day_data], axis = 1)

In [24]:
dock_id_series = final_df['dock_id']

In [25]:
final_df.drop(['dock_id'], axis = 1, inplace = True)

In [26]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster
0,12,2,12,40.683826,-73.976323,62,-1.01,0,2,3
1,12,2,12,40.696089,-73.978034,19,-1.01,0,2,0
2,12,2,12,40.686768,-73.959282,19,-1.01,0,1,0
3,12,2,12,40.692395,-73.993379,24,-1.01,0,2,3
4,12,2,12,40.698399,-73.980689,19,-1.01,0,2,0
...,...,...,...,...,...,...,...,...,...,...
324,12,2,12,40.805726,-73.936322,33,-1.01,0,2,2
325,12,2,12,40.801307,-73.939817,29,-1.01,0,0,2
326,12,2,12,40.804555,-73.939686,36,-1.01,0,0,2
327,12,2,12,40.801194,-73.950074,31,-1.01,0,2,0


In [27]:
predictions = model.best_estimator_.predict(final_df)

In [28]:
final_df['predictions'] = predictions
final_df['dock_id'] = dock_id_series


In [29]:
final_df['avail_bikes_proportion'] = final_df['predictions']/final_df['tot_docks']

In [30]:
final_df

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion
0,12,2,12,40.683826,-73.976323,62,-1.01,0,2,3,27.357524,83,0.441250
1,12,2,12,40.696089,-73.978034,19,-1.01,0,2,0,9.056517,119,0.476659
2,12,2,12,40.686768,-73.959282,19,-1.01,0,1,0,3.939524,120,0.207343
3,12,2,12,40.692395,-73.993379,24,-1.01,0,2,3,20.074048,143,0.836419
4,12,2,12,40.698399,-73.980689,19,-1.01,0,2,0,16.099226,144,0.847328
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,12,2,12,40.805726,-73.936322,33,-1.01,0,2,2,10.558578,3505,0.319957
325,12,2,12,40.801307,-73.939817,29,-1.01,0,0,2,5.144976,3506,0.177413
326,12,2,12,40.804555,-73.939686,36,-1.01,0,0,2,11.518306,3507,0.319953
327,12,2,12,40.801194,-73.950074,31,-1.01,0,2,0,11.935893,3509,0.385029


In [31]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [32]:
low_bike_threshold = 1/3
high_bike_threshold = 2/3
max_distance = 2
max_bikes_rebalanced = 500
min_cargo_size = 3

In [33]:
data_low = final_df[final_df['avail_bikes_proportion'] <= low_bike_threshold]

In [34]:
data_high = final_df[final_df['avail_bikes_proportion'] >= high_bike_threshold]

In [35]:
data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-35-d746bb208678>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((low_bike_threshold - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [36]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')

<ipython-input-36-d9c66eeb9a9d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - high_bike_threshold) * data_high['tot_docks']).astype('int')


In [37]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [38]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [39]:
rebalancing_dict = {}

In [40]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [41]:
bikes_rebalanced = 0
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < max_distance:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            bikes_rebalanced += change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [42]:
rebalancing_dict

{(3161, 3457): 8,
 (3161, 456): 6,
 (3161, 3318): 2,
 (3295, 3367): 6,
 (3295, 3318): 4,
 (3295, 3296): 4,
 (3164, 524): 11,
 (3164, 3132): 3,
 (3140, 3132): 3,
 (3140, 3296): 2,
 (3140, 3136): 5,
 (3140, 3459): 4,
 (3167, 3233): 5,
 (3167, 3458): 4,
 (3167, 3235): 3,
 (3146, 3459): 1,
 (3146, 3387): 5,
 (3146, 3379): 4,
 (3146, 3141): 2,
 (3437, 3502): 4,
 (3294, 3494): 3,
 (3294, 3500): 1,
 (3294, 3312): 1,
 (3294, 3338): 1,
 (3165, 164): 1,
 (3148, 3130): 2,
 (3452, 3093): 6,
 (3452, 3092): 3,
 (3375, 516): 4,
 (3375, 2017): 2,
 (3075, 3092): 3,
 (3075, 2001): 5,
 (3443, 3258): 5,
 (3443, 325): 2,
 (454, 267): 3,
 (454, 2012): 1,
 (454, 491): 1,
 (3077, 3090): 4,
 (3077, 3096): 2,
 (3358, 3409): 6,
 (448, 462): 6,
 (3044, 3416): 4,
 (3044, 437): 2,
 (365, 3409): 2,
 (365, 3373): 4,
 (3306, 3373): 3,
 (3306, 390): 3,
 (3310, 3477): 6,
 (3354, 3399): 6,
 (3365, 390): 3,
 (3365, 3324): 3,
 (3308, 337): 5,
 (3423, 3324): 3,
 (3423, 3335): 2,
 (389, 2001): 1,
 (389, 414): 4,
 (3124, 3123

In [43]:
sorted_rebalancing = dict(sorted(rebalancing_dict.items(), key=lambda x: x[1], reverse = True))

In [44]:
filtered_rebalancing = {key: value for key, value in sorted_rebalancing.items() if value >= min_cargo_size}

In [45]:
filtered_rebalancing

{(3164, 524): 11,
 (3161, 3457): 8,
 (3161, 456): 6,
 (3295, 3367): 6,
 (3452, 3093): 6,
 (3358, 3409): 6,
 (448, 462): 6,
 (3310, 3477): 6,
 (3354, 3399): 6,
 (3140, 3136): 5,
 (3167, 3233): 5,
 (3146, 3387): 5,
 (3075, 2001): 5,
 (3443, 3258): 5,
 (3308, 337): 5,
 (3124, 3123): 5,
 (3117, 3463): 5,
 (3295, 3318): 4,
 (3295, 3296): 4,
 (3140, 3459): 4,
 (3167, 3458): 4,
 (3146, 3379): 4,
 (3437, 3502): 4,
 (3375, 516): 4,
 (3077, 3090): 4,
 (3044, 3416): 4,
 (365, 3373): 4,
 (389, 414): 4,
 (3396, 3340): 4,
 (436, 243): 4,
 (416, 3371): 4,
 (400, 3467): 4,
 (174, 168): 4,
 (3164, 3132): 3,
 (3140, 3132): 3,
 (3167, 3235): 3,
 (3294, 3494): 3,
 (3452, 3092): 3,
 (3075, 3092): 3,
 (454, 267): 3,
 (3306, 3373): 3,
 (3306, 390): 3,
 (3365, 390): 3,
 (3365, 3324): 3,
 (3423, 3324): 3,
 (3449, 539): 3,
 (3456, 3112): 3,
 (3321, 337): 3,
 (3050, 3429): 3,
 (396, 3335): 3,
 (397, 3404): 3,
 (291, 3467): 3,
 (3048, 3419): 3,
 (3315, 260): 3,
 (3241, 3455): 3}

In [46]:
filtered_bikes_rebalanced = 0
final_rebalancing_dict = {}
for k, v in filtered_rebalancing.items():
    if filtered_bikes_rebalanced < max_bikes_rebalanced:
        final_rebalancing_dict[k] = v
        filtered_bikes_rebalanced += v

In [47]:
rebalancing_df = pd.DataFrame(final_rebalancing_dict.items(), columns = ['dock_ids', 'num_bikes'])

In [48]:
rebalancing_df[['dock_id_receive', 'dock_id_give']] = rebalancing_df['dock_ids'].tolist()
rebalancing_df.drop(['dock_ids'], axis = 1, inplace = True)

In [49]:
data_df = final_df[['dock_id', 'latitude', 'longitude']]

In [50]:
new_docks = new[['dock_id', 'dock_name']]

In [51]:
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_receive', 'longitude': 'longitude_receive'})
rebalancing_df = rebalancing_df.merge(data_df, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'latitude': 'latitude_give', 'longitude': 'longitude_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [52]:
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_receive', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_receive'})
rebalancing_df = rebalancing_df.merge(new_docks, how = 'left', left_on = 'dock_id_give', right_on = 'dock_id').rename(
    columns = {'dock_name': 'dock_name_give'})
rebalancing_df.drop(['dock_id_x', 'dock_id_y'], axis = 1, inplace = True)

In [53]:
GREEN_RGB = [0, 255, 0, 150]
RED_RGB = [240, 100, 0, 150]

# Specify a deck.gl ArcLayer
arc_layer = pdk.Layer(
    "ArcLayer",
    data = rebalancing_df,
    get_width="num_bikes",
    get_source_position=["longitude_give", "latitude_give"],
    get_target_position=["longitude_receive", "latitude_receive"],
    get_tilt=15,
    get_source_color=GREEN_RGB,
    get_target_color=RED_RGB,
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(latitude=40.74, longitude=-74, bearing=290, pitch=50, zoom=12)


TOOLTIP_TEXT = {"html": "{num_bikes} bikes need rebalancing from<br />{dock_name_give} to {dock_name_receive}"}
r = pdk.Deck(arc_layer, initial_view_state=view_state, tooltip=TOOLTIP_TEXT, map_style = 'light')
r.to_html("monday_12pm_rebalancing.html")
with open('monday_12pm_rebalancing.json', 'w') as outfile:
    json.dump(r.to_json(), outfile)

In [54]:
bikes_rebalanced

308

In [55]:
filtered_bikes_rebalanced

231

In [56]:
data_low['deficit'].sum()

713

In [57]:
data_high['surplus'].sum()

499

In [58]:
data_low.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,deficit
151,12,2,12,40.780184,-73.977285,59,-1.01,0,1,2,3.971548,3161,0.067314,16
182,12,2,12,40.791270,-73.964839,59,-1.01,0,1,2,5.765762,3295,0.097725,14
154,12,2,12,40.777057,-73.978985,67,-1.01,0,1,2,8.497000,3164,0.126821,14
143,12,2,12,40.771404,-73.953517,55,-1.01,0,4,0,4.437405,3140,0.080680,14
156,12,2,12,40.779668,-73.980930,55,-1.01,0,1,2,6.681357,3167,0.121479,12


In [59]:
data_high.head(5)

,month,num_day,hour,latitude,longitude,tot_docks,tmp,rain,weekday_cluster,weekend_cluster,predictions,dock_id,avail_bikes_proportion,surplus
303,12,2,12,40.724947,-74.001659,42,-1.01,0,2,0,39.315762,3467,0.936090,11
78,12,2,12,40.746920,-74.004519,47,-1.01,0,4,0,42.083595,462,0.895396,11
8,12,2,12,40.739713,-73.994564,47,-1.01,0,5,3,41.887616,168,0.891226,11
48,12,2,12,40.716021,-73.999744,43,-1.01,0,2,3,39.533929,355,0.919394,11
83,12,2,12,40.755273,-73.983169,57,-1.01,0,5,1,48.972833,524,0.859173,11


In [84]:
table = rebalancing_df[['dock_name_give', 'dock_name_receive', 'num_bikes']]
table = table.rename(columns = {'dock_name_give': 'dock_origin', 'dock_name_receive': 'dock_destination'})

In [86]:
table

,dock_origin,dock_destination,num_bikes
0,W 43 St & 6 Ave,Columbus Ave & W 72 St,11
1,E 58 St & Madison Ave,W 76 St & Columbus Ave,8
2,E 53 St & Madison Ave,W 76 St & Columbus Ave,6
3,5 Ave & E 103 St,Central Park W & W 96 St,6
4,N 6 St & Bedford Ave,Bayard St & Leonard St,6
5,Bergen St & Smith St,Garfield Pl & 8 Ave,6
6,W 22 St & 10 Ave,W 37 St & 10 Ave,6
7,39 St & 2 Ave - Citi Bike HQ at Industry City,14 St & 7 Ave,6
8,7 St & 3 Ave,3 St & Prospect Park West,6
9,5 Ave & E 63 St,1 Ave & E 78 St,5


In [82]:
table_html = table.to_html()

text_file = open("rebalancing_table.html", "w")
text_file.write(table_html)
text_file.close()

AttributeError: 'NoneType' object has no attribute 'to_html'